In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
# selenium을 사용하여 데이터 가져오기
url = 'https://www.destinypedia.com/The_Maraid'

driver = webdriver.Chrome()
time.sleep(1)
driver.get(url)

title = []
description = []

for page_number in range(1, 215):
    
    page = str(page_number)

    items = driver.find_element(By.LINK_TEXT, page)
    items.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    bhc_df = web_search(soup, 1)

    if page_number % 10 == 0:
        items = driver.find_element(By.CSS_SELECTOR, 'a[alt="다음"]')
        items.click()
        time.sleep(1)

bhc_df

In [ ]:
def web_search(soup, sw):
    df = pd.DataFrame()
    
    global store_list, store_list_address

    for line in soup.find_all('tr'):

        store_name = line.find('p').text.strip()    # 첫 번째 p 태그의 텍스트를 가져옴. 공백 제거는 꼭 해줘야하는건지 몰라서 넣어둠.
        if store_name == '':                        # store_name이 ''이면(None과 다름!)
            store_name = line.find('div', style="color: #000000; font-weight: bold;font-size: 16px;").text.strip()  # div 태그로 찾아서 텍스트를 가져옴
        store_list.append(store_name)

        store_address = line.find('span', class_='bold').next_sibling.strip().split(': ')[1]    # bold를 찾고 공백을 지운 후 그 다음 요소를 ':'로 분할 후 두 번째 것을 가져옴
        store_list_address.append(store_address)

    # print(store_list)
    # print(store_list_address)

    df['store_name'] = store_list
    if sw == 1:
        df['Address'] = store_list_address
    else:
        df['Point'] = store_list_address
    return df

In [ ]:
# selenium을 사용하여 데이터 가져오기
url = 'https://www.bhc.co.kr/location/map.asp'

driver = webdriver.Chrome()
time.sleep(1)
driver.get(url)

store_list = []
store_list_address = []

for page_number in range(1, 215):
    
    page = str(page_number)

    items = driver.find_element(By.LINK_TEXT, page)
    items.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    bhc_df = web_search(soup, 1)

    if page_number % 10 == 0:
        items = driver.find_element(By.CSS_SELECTOR, 'a[alt="다음"]')
        items.click()
        time.sleep(1)

bhc_df

# Code from JP

In [4]:
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import pandas as pd
import re

In [6]:
url_weblore = 'https://www.destinypedia.com/Weblore:Joker%27s_Wild'

html = urlopen(url_weblore)
bs = BeautifulSoup(html, 'html.parser')

html_full = bs.select('.mw-parser-output')

In [7]:
html_full = bs.select_one('.mw-parser-output')

In [8]:
list_title = []
list_text = []
text = ''
title_flag = False

for child in html_full:
    if child.name == 'h2':
        
        if title_flag == True:
            list_text.append(text)
            text = ''
        list_title.append(child.text)
        title_flag = True
    elif title_flag == True and child.name == 'p':
        text += child.text

In [ ]:
for i in range(len(list_season)):
    ## 데이터 프레임을 만들기 위해 필요한 변수 선언
    list_expension = []
    list_title = []
    list_text = []
    text = ''
    title_flag = False
    
    ## url사용시 에러나는부분 고처쓰기위한 부분
    list_season[i] = re.sub("'",'%27',list_season[i])
    list_season[i] = re.sub(" ",'_',list_season[i])
    
    ## beautiful soup 객체 선언
    url_season = url_weblore_root + ':' + list_season[i]
    html = urlopen(url_season)
    bs = BeautifulSoup(html, 'html.parser')
    
    ## class명으로 태그 선택
    html_full = bs.select_one('.mw-parser-output')
    
    ## 태그 안에 들어가 제목과 내용 분류해서 list에 넣기
    for child in html_full:
        if child.name == 'h2':
            ## 다음 제목으로 넘어가게 되면 이전 내용을 list에 저장 후 내용 초기화
            if title_flag == True:
                list_text.append(text)
                text = ''
            ## 제목이 들어있는 태그 만나면 제목리스트에 저장
            list_title.append(child.text)
            title_flag = True
        ## html_full을 처음 선택하면 있는 p태그를 건너뛰고 다음 p태그부터 저장할 내용 연결
        elif title_flag == True and child.name == 'p':
            text += child.text
    
    ## 제목 리스트의 마지막에 references제거         
    list_title.remove('References')
    
    ## url사용하기 위해 고친 부분 다시 원상태로 만들기
    list_season[i] = re.sub("%27","'",list_season[i])
    list_season[i] = re.sub("_",' ',list_season[i])
    
    ## 확장팩 리스트 만들기
    for j in range(len(list_title)):
        list_expension.append(list_season[i])
    
    ## 확인용    
    print(list_expension)
    print(list_title)
    print(list_text)
       
    ## 만든 리스트로 데이터 프레임 만들어서 csv파일로 저장 
    df_data = pd.DataFrame({'expension_name': list_expension, 'title': list_title, 'text':list_text})
    df_data.to_csv('df_' + list_season[i] + '.csv', index= False)

# For 'The Maraid'

In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url_root = 'https://www.destinypedia.com/The_Maraid'
html = urlopen(url_root)
soup = BeautifulSoup(html, 'html.parser')
html_full = soup.select_one('.mw-parser-output')

title_flag = False
title_the_maraid = []
text_the_maraid = []
text = ''

## 태그 안에 들어가 제목과 내용 분류해서 list에 넣기
for child in html_full:
    if child.name == 'h3':
        ## 다음 제목으로 넘어가게 되면 이전 내용을 list에 저장 후 내용 초기화
        if title_flag == True:
            text_the_maraid.append(text)
            text = ''
        ## 제목이 들어있는 태그 만나면 제목리스트에 저장
        title_the_maraid.append(child.text)
        title_flag = True
    ## html_full을 처음 선택하면 있는 p태그를 건너뛰고 다음 p태그부터 저장할 내용 연결
    elif title_flag == True and child.name == 'p':
        text += child.text
# 마지막에 'h3'을 만나지 못해 저장되지 않은 text 저장
text_the_maraid.append(text)

print(title_the_maraid)
print(text_the_maraid)
print(len(title_the_maraid))
print(len(text_the_maraid))

['The Maraid, Book VII, Chapter 10[edit]', 'The Maraid, Book VIII, Chapter 1[edit]', 'The Maraid, Book VIII, Chapter 2[edit]', 'The Maraid, Book VIII, Chapter 3[edit]', 'The Maraid, Book VIII, Chapter 4[edit]', 'The Maraid, Book VIII, Chapter 5[edit]', 'The Maraid, Book VIII, Chapter 6[edit]', 'The Maraid, Book VIII, Chapter 7[edit]', 'The Maraid, Book VIII, Chapter 8[edit]', 'The Maraid, Book VIII, Chapter 9[edit]', 'The Maraid, Book VIII, Chapter 10[edit]', 'The Hunt for Skolas Bounty (2) description[edit]', 'Gone to Ground Bounty (7) description[edit]', 'Gone to Ground Bounty (6) description[edit]', 'Gone to Ground Bounty (5) description[edit]', 'Gone to Ground Bounty (2) description[edit]', 'Gone to Ground Bounty (1) description[edit]', 'Gone to Ground Bounty (3) description[edit]', 'The Kell of Kells Bounty (2) description[edit]', 'The Kell of Kells Bounty (5) description[edit]', 'The Kell of Kells Bounty (3) description[edit]', 'The Kell of Kells Bounty (1) description[edit]']
["

In [3]:
the_maraid = pd.DataFrame([title_the_maraid, text_the_maraid])
the_maraid

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,"The Maraid, Book VII, Chapter 10[edit]","The Maraid, Book VIII, Chapter 1[edit]","The Maraid, Book VIII, Chapter 2[edit]","The Maraid, Book VIII, Chapter 3[edit]","The Maraid, Book VIII, Chapter 4[edit]","The Maraid, Book VIII, Chapter 5[edit]","The Maraid, Book VIII, Chapter 6[edit]","The Maraid, Book VIII, Chapter 7[edit]","The Maraid, Book VIII, Chapter 8[edit]","The Maraid, Book VIII, Chapter 9[edit]",...,Gone to Ground Bounty (7) description[edit],Gone to Ground Bounty (6) description[edit],Gone to Ground Bounty (5) description[edit],Gone to Ground Bounty (2) description[edit],Gone to Ground Bounty (1) description[edit],Gone to Ground Bounty (3) description[edit],The Kell of Kells Bounty (2) description[edit],The Kell of Kells Bounty (5) description[edit],The Kell of Kells Bounty (3) description[edit],The Kell of Kells Bounty (1) description[edit]
1,Abstract:\nThe transmission was broadcasted on...,"Abstract:\nAfter the Scatter, the frontrunner ...",Abstract:\nWhat Peekis' assault lacked in fine...,Abstract:\nWith Skolas and Parixas still fight...,Abstract:\nAfter a string of defeats—at Amethy...,"Abstract:\nUnder Skolas' vassal Pirsis, called...",Abstract:\nWeksis' attack may have been unsucc...,"Abstract:\nFinally, Beltrik, the Veiled left t...","Abstract:\nIn desperation, Skolas personally l...",Abstract:\nVariks of the House of Judgment dec...,...,"""After the Scatter, many would-be kells squabb...","""Prince Uldren discovered deep flaws in the Wo...","""Led by Drevis, the Silent Fang evaded the Web...","""After the Battle of Iris, the united Wolves s...","""Paladin Abra Zire pursued the Silent Fang fro...","""When the battle at Iris faded, it was not Sko...","""Prince Uldren's Crows discovered the Wolves h...","""Paladin Rife forced Drevis's ships directly i...","""Paladin Rife set course for Vesta with her ca...","""The Hildian Campaign had two purposes: to bre..."


In [4]:
the_maraid = the_maraid.transpose()
the_maraid

,0,1
0,"The Maraid, Book VII, Chapter 10[edit]",Abstract:\nThe transmission was broadcasted on...
1,"The Maraid, Book VIII, Chapter 1[edit]","Abstract:\nAfter the Scatter, the frontrunner ..."
2,"The Maraid, Book VIII, Chapter 2[edit]",Abstract:\nWhat Peekis' assault lacked in fine...
3,"The Maraid, Book VIII, Chapter 3[edit]",Abstract:\nWith Skolas and Parixas still fight...
4,"The Maraid, Book VIII, Chapter 4[edit]",Abstract:\nAfter a string of defeats—at Amethy...
5,"The Maraid, Book VIII, Chapter 5[edit]","Abstract:\nUnder Skolas' vassal Pirsis, called..."
6,"The Maraid, Book VIII, Chapter 6[edit]",Abstract:\nWeksis' attack may have been unsucc...
7,"The Maraid, Book VIII, Chapter 7[edit]","Abstract:\nFinally, Beltrik, the Veiled left t..."
8,"The Maraid, Book VIII, Chapter 8[edit]","Abstract:\nIn desperation, Skolas personally l..."
9,"The Maraid, Book VIII, Chapter 9[edit]",Abstract:\nVariks of the House of Judgment dec...


In [6]:
the_maraid.columns = ['title', 'description']
the_maraid

,title,description
0,"The Maraid, Book VII, Chapter 10[edit]",Abstract:\nThe transmission was broadcasted on...
1,"The Maraid, Book VIII, Chapter 1[edit]","Abstract:\nAfter the Scatter, the frontrunner ..."
2,"The Maraid, Book VIII, Chapter 2[edit]",Abstract:\nWhat Peekis' assault lacked in fine...
3,"The Maraid, Book VIII, Chapter 3[edit]",Abstract:\nWith Skolas and Parixas still fight...
4,"The Maraid, Book VIII, Chapter 4[edit]",Abstract:\nAfter a string of defeats—at Amethy...
5,"The Maraid, Book VIII, Chapter 5[edit]","Abstract:\nUnder Skolas' vassal Pirsis, called..."
6,"The Maraid, Book VIII, Chapter 6[edit]",Abstract:\nWeksis' attack may have been unsucc...
7,"The Maraid, Book VIII, Chapter 7[edit]","Abstract:\nFinally, Beltrik, the Veiled left t..."
8,"The Maraid, Book VIII, Chapter 8[edit]","Abstract:\nIn desperation, Skolas personally l..."
9,"The Maraid, Book VIII, Chapter 9[edit]",Abstract:\nVariks of the House of Judgment dec...


In [7]:
the_maraid.to_json('the_maraid.json', orient='records', lines=False)